<a href="https://colab.research.google.com/github/ngolemai020405/Tuan5/blob/main/Tuan5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

means = [[2, 2], [9, 2], [4, 9]]
cov = [[2, 0], [0, 2]]
n_samples = 500
n_cluster = 3
X0 = np.random.multivariate_normal(means[0], cov, n_samples)
X1 = np.random.multivariate_normal(means[1], cov, n_samples)
X2 = np.random.multivariate_normal(means[2], cov, n_samples)
X = np.concatenate((X0, X1, X2), axis = 0)

plt.xlabel('x')
plt.ylabel('y')
plt.plot(X[:,0], X[:,1],'bo',markersize=5)
plt.plot()
plt.show()

def kmeans_init_centers(X,n_cluster):
  return X[np.random.choice(X.shape[0], n_cluster,replace=False)]

def kmeans_predict_labels(X,centers):
  D=cdist(X,centers)
  return np.argmin(D,axis=1)

def kmeans_update_centers(X, labels, n_cluster):
  centers = np.zeros((n_cluster, X.shape[1]))
  for k in range(n_cluster):

  Xk = X[labels == k, :]

  centers[k,:] = np.mean(Xk, axis = 0)
return centers